# THESIS - Preprocessing 


Four datasets are going to be used : 

- RNA-seq for HEK293 cells (Sun et al) --> HEK  
- RNA-seq for HepG2 (Wold, ENCODE) --> Hep 
- ENCODE eCLIP for protein-RNA interactions (Wold, ENCODE)
- miCLIP for m6A modifications (HEK293 cells, Linder et al)

Firstly, RNA-seq data is going to be filtered and combined in order to obtain a dataset which contains genes that are expressed in both HEK293 and HepG2 cell lines. 


During the preprocessing, dataframes are utilized and for this purpose, the pandas library is particularly effective. 

In [27]:
import pandas as pd 
import numpy as np 
from pybedtools import BedTool
import pybedtools
#import pybiomart
import scanpy as sc
import os , sys

# 1. The dataset RNA-seq for HEK293 cells (Sun et al)
The dataset RNA-seq for HEK293 cells (Sun et al) is uploaded and the dataframe dfHek is produced. 
It is noticeable from the function len() that the number of elements contained is equal to 57905. 
It is necessary to eliminate the transcripts that are not expressed: the amount of elements after the stripping(?) is equal to 32396.   
Notice that the two RNA-seq files have different genome versions, it is necesssary to lift one or the other, in order for them to be in the same version. This is going to be achieved by an R script and the use of the package useMart(): maybe copy the script here. 
I have decided to lift HEK293 to the version hg38/GRCh38 and I am going to use the lifted version for the comparison. 


In [2]:
dfHEK = pd.read_excel("HEK293.xlsx")

In [4]:
#check if the gene_id actually changed correctly: at position 22859 --> 'ENSG00000005955'
dfHEK.loc[22859]

gene_id                                                 ENSG00000005955
length                                                            45542
HEK293NK-SEQ1                                                      3534
HEK293NK-SEQ1_RPKM                                                 6.65
HEK293NK-SEQ2                                                      3664
HEK293NK-SEQ2_RPKM                                                 6.54
HEK293NK-SEQ3                                                      4111
HEK293NK-SEQ3_RPKM                                                 6.94
HEK293-SEQ1                                                        3181
HEK293-SEQ1_RPKM                                                   6.56
HEK293-SEQ2                                                        3078
HEK293-SEQ2_RPKM                                                    6.3
HEK293-SEQ3                                                        3267
HEK293-SEQ3_RPKM                                                

In [5]:
len(dfHEK)

57905

In [3]:
#dropping transcripts without expression.
dfHEK.columns = dfHEK.columns.str.replace('-', '_')
dfHEK = dfHEK[(dfHEK.HEK293NK_SEQ1 != 0.00) | (dfHEK.HEK293NK_SEQ2 != 0.00)]
len(dfHEK)

32396

In [4]:
dfVersion = pd.read_excel("inconsistenciesENSEMBL_noNaN.xlsx", usecols = "B, C")

dictionary = pd.Series(dfVersion['ensembl_gene_id.y'].values,index=dfVersion['ensembl_gene_id.x']).to_dict()

#Problem: the dictionary has duplicates: for example ENSG00000257341 is linked both to 'ENSG00000257341' and 'ENSG00000213145'--> the fuction replace is 
#using the same ENSG00000257341, ignoring the other. 

In [5]:
#just keep the lifted ids.--> will be just protein coding genes since we are using ccds as unique key.   
#dfVersion = pd.read_excel("incostintenciesENSEMBL.xlsx")
dfHEK = dfHEK.replace({"gene_id": dictionary})

In [9]:
#check if the gene_id actually changed correctly: at position 22859 --> 'ENSG00000278311' --> it's right 
dfHEK.loc[22859]

gene_id                                                 ENSG00000278311
length                                                            45542
HEK293NK_SEQ1                                                      3534
HEK293NK_SEQ1_RPKM                                                 6.65
HEK293NK_SEQ2                                                      3664
HEK293NK_SEQ2_RPKM                                                 6.54
HEK293NK_SEQ3                                                      4111
HEK293NK_SEQ3_RPKM                                                 6.94
HEK293_SEQ1                                                        3181
HEK293_SEQ1_RPKM                                                   6.56
HEK293_SEQ2                                                        3078
HEK293_SEQ2_RPKM                                                    6.3
HEK293_SEQ3                                                        3267
HEK293_SEQ3_RPKM                                                

In [10]:
#dataframe without unexpressed genes and with genome version hg38 
dfHEK


,gene_id,length,HEK293NK_SEQ1,HEK293NK_SEQ1_RPKM,HEK293NK_SEQ2,HEK293NK_SEQ2_RPKM,HEK293NK_SEQ3,HEK293NK_SEQ3_RPKM,HEK293_SEQ1,HEK293_SEQ1_RPKM,HEK293_SEQ2,HEK293_SEQ2_RPKM,HEK293_SEQ3,HEK293_SEQ3_RPKM,GeneSymbol,KO,GO
1,ENSG00000227232,15444,705,4.77,812,5.21,1121,6.80,732,5.43,690,5.07,804,5.39,WASH7P,K18461,_
7,ENSG00000238009,44272,1,0.01,2,0.01,0,0.00,1,0.01,3,0.02,1,0.01,RP11-34P13.7,_,_
9,ENSG00000233750,3812,1,0.01,0,0.00,0,0.00,0,0.00,0,0.00,1,0.01,CICP27,K12581,_
10,ENSG00000237683,4479,4,0.04,10,0.09,7,0.06,9,0.09,12,0.12,11,0.10,AL627309.1,_,_
14,ENSG00000241860,32389,13,0.05,4,0.01,2,0.01,14,0.06,6,0.02,2,0.01,RP11-34P13.13,_,_
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57433,ENSG00000231341,855,2,0.06,1,0.03,2,0.05,0,0.00,1,0.03,1,0.03,VDAC1P6,K05862,_
57434,ENSG00000235001,1220,19,0.19,7,0.07,12,0.11,8,0.09,9,0.10,8,0.08,EIF4A1P2,K03257,_
57589,ENSG00000215414,741,17,0.29,9,0.14,17,0.26,8,0.15,10,0.18,10,0.17,PSMA6P1,K02730,_
57691,ENSG00000185275,243,636,64.62,682,65.72,317,28.92,361,40.24,394,43.54,355,35.78,CD24P4,K06469,_


'''tried to imput in the R script just the expressed genes but the problem persists: 
the actual length of the gene_id column in HEK293 is 57 905 (32396 now )but 
once the merge function is applied in the script, only 29 285 elements are kept. 
NB: out37 and out38 are containing 29 107 and 28 126 CCDS respectively, 
which means that not all of the gene_ids contained in the file HEK293 have a corresponding CCDS in Mart37 or 38. 
I have checked for duplicates in HEK293 and there are just 2. 
The problem must be related to the number of CCDS.
'The Consensus CDS (CCDS) project is a collaborative effort to identify a core set of human and mouse protein CODING REGIONS 
that are consistently annotated and of high quality. 
The long term goal is to support convergence towards a standard set of gene annotations.[https://www.ncbi.nlm.nih.gov/projects/CCDS/CcdsBrowse.cgi]''''



# 2. The dataset RNA-seq for HepG2 (Wold, ENCODE)
The dataset RNA-seq for HepG2 (Wold, ENCODE) is uploaded and the dataframe dfHep is produced. The dataset is processed analogously, removing non-expressed transcripts. In this case the original number of values is 207507, whereas the elements after the elimination correspond to 96044 gene identifiers. (?)  
PS: SEEMS LIKE gene_ids ARE NOT ONLY IN THE FORMAT ENSEMBL, IS THIS A PROBLEM? --> no


In [6]:
dfHep = pd.read_excel("HepG2.xlsx")
print(len(dfHep))
dfHep

207507


,transcript_id,gene_id,length,effective_length,expected_count,TPM,FPKM,IsoPct,posterior_mean_count,posterior_standard_deviation_of_count,pme_TPM,pme_FPKM,IsoPct_from_pme_TPM,TPM_ci_lower_bound,TPM_ci_upper_bound,TPM_coefficient_of_quartile_variation,FPKM_ci_lower_bound,FPKM_ci_upper_bound,FPKM_coefficient_of_quartile_variation
0,10904,10904,93,0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,12954,12954,94,0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,12956,12956,72,0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,12958,12958,82,0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,12960,12960,73,0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207502,tSpikein_ERCC-00165,gSpikein_ERCC-00165,872,773,182.0,4.79,5.10,100.0,182.0,0.0,4.70,5.11,100.0,4.027980,5.379590,0.049978,4.379740,5.849960,0.049976
207503,tSpikein_ERCC-00168,gSpikein_ERCC-00168,1024,925,1.0,0.02,0.02,100.0,1.0,0.0,0.04,0.05,100.0,0.001247,0.102962,0.475355,0.001355,0.111972,0.475281
207504,tSpikein_ERCC-00170,gSpikein_ERCC-00170,1023,924,68.0,1.50,1.60,100.0,68.0,0.0,1.48,1.61,100.0,1.146680,1.843870,0.080920,1.243590,2.001790,0.080810
207505,tSpikein_ERCC-00171,gSpikein_ERCC-00171,505,406,7125.0,357.14,380.37,100.0,7125.0,0.0,348.37,378.75,100.0,340.237000,356.454000,0.007965,370.223000,387.840000,0.007956


In [7]:
#dropping all transcripts that are not expressed 
dfHep = dfHep[dfHep.TPM != 0.00]
len(dfHep)
#check the shape of the set

96044

# 3. Identify an intersection of genes expressed in HEK293 and HepG2 cell lines 
The objective is to consider the RNA-seq datasets comparing the genes that are expressed in both cell lines. 
For this reason, it is crucial to observe the gene_ids formats in the two examples and ensure their compatibility. 
In this very case, the identifiers contained in the dataset HepG2 are complemented with the model's version, which is not specified in HEK293: it is essential to level out these conceptual differences..   

In [8]:
#duplicating gene_id to manipulate it
dfHep['gene_id_new'] = dfHep['gene_id']

#for the comparison, since in dfHep there are more detailed gene_ids containing also the version of the model, I have decided to remove the last part of the id. 
dfHep['gene_id_new']= dfHep.gene_id_new.str.split('.').str[0]

/tmp/ipykernel_89151/1669813914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHep['gene_id_new'] = dfHep['gene_id']
/tmp/ipykernel_89151/1669813914.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHep['gene_id_new']= dfHep.gene_id_new.str.split('.').str[0]


The two sets are prepared to be intersected : the final intersection comprises of 20875 [14474] --> 20953 elements.  

In [9]:
#intersecting the two columns so that in set3 there are just the ids in common 
listHEK = dfHEK['gene_id'].to_list() 
listHep = dfHep['gene_id_new'].to_list()
set_common_genes = set(listHEK).intersection(set(listHep))
len(set_common_genes)

20954

# 4. CLIP data 

At this point it is possible to upload the CLIP files, in this case eCLIP and miCLIP files. The aim is to just keep the genes that are contained in the set of common expressed genes from the Ref_Seq: for this aim, functions are going to be implemented,they should be as general as possible.  
To note: in the eCLIP file, there are no ENSEMBL ids, just the locus --> it is needed to compare them to get the info. 

In [47]:
#the annotations contain more than just one ensembl gene id per locus, this function explodes the field in which the ids are 
#stored so that they are more easily accessible for compare_with_common_genes
def explode_annotations(ann) :
    annots = pd.Series(ann.split(';'))
    annots = annots.str.split('|', expand = True)
    annots.columns = ['gene_id', 'gene_name', 'gene_type']
    annots.gene_id= annots.gene_id.str.split('.').str[0]
    return annots

In [48]:
#takes a long time --> converted to a set instead of a dataframe so that it's faster. 
#this function compares the names stored in a .bed file row with the set of common genes previously produced by intersection. 
def compare_with_common_genes(feature, genes_to_compare): 
    df = explode_annotations(feature) 
    intersection = set(df.gene_id).intersection(genes_to_compare) 
    if len(intersection) > 0 : 
        return True
    return False
    

Once the functions are implemented, the files are going to be processed and the common ids extracted. The miCLIP file is processed first. 

In [49]:
miCLIP = BedTool("miCLIP.bed")

df_miCLIP = miCLIP.to_dataframe(disable_auto_names=True, header= None)


miCLIP.head()

chr10	98416554	98416555	B045_1-4_chr10_r_c635[k=4][m=2]_AGACT	1	-	chr10	98416198	98446935	ENSG00000107521.20|HPS1|mRNA;ENSG00000264610.1|MIR4685|miRNA	.	-	1
 chr10	99330834	99330835	B045_1-4_chr10_f_c695[k=4][m=2]_GGACT	1	+	chr10	99329356	99394330	ENSG00000119946.12|CNNM1|mRNA	.	+	1
 chr10	99611239	99611240	B045_1-4_chr10_r_c643[k=4][m=2]_GAACG	1	-	chr10	99610522	99620609	ENSG00000155287.11|SLC25A28|mRNA	.	-	1
 chr10	99743263	99743264	B045_1-4_chr10_f_c698[k=4][m=2]_TGACA	1	+	chr10	99659509	99756134	ENSG00000198018.7|ENTPD7|mRNA;ENSG00000119929.13|CUTC|mRNA	.	+	1
 chr10	99755703	99755704	B045_1-4_chr10_f_c699[k=11][m=2]_GGACT	1	+	chr10	99659509	99756134	ENSG00000198018.7|ENTPD7|mRNA;ENSG00000119929.13|CUTC|mRNA	.	+	1
 chr10	99755836	99755837	B045_1-4_chr10_f_c701[k=8][m=2]_GGACA	1	+	chr10	99659509	99756134	ENSG00000198018.7|ENTPD7|mRNA;ENSG00000119929.13|CUTC|mRNA	.	+	1
 chr10	99876942	99876943	B045_1-4_chr10_r_c647[k=6][m=2]_GGACC	1	-	chr10	99875577	100009947	ENSG00000107554.17|DNMBP|

In [50]:
#checking the output of explode_annotations
explode_annotations(df_miCLIP[9][0])

,gene_id,gene_name,gene_type
0,ENSG00000107521,HPS1,mRNA
1,ENSG00000264610,MIR4685,miRNA


In [52]:
#here the fuction is applied to all of the rows of the dataframe, filling a new column 'in intersection', to make it easy to then filter the df
df_miCLIP['gene'] = df_miCLIP[9]
df_miCLIP['in intersection'] =  df_miCLIP.gene.map(lambda x: compare_with_common_genes(x, genes_to_compare=set_common_genes))
df_miclip_ok = df_miCLIP[df_miCLIP['in intersection'] == True]


In [53]:
len(df_miCLIP)

9278

In [54]:
len(df_miclip_ok)

9087

#starting to work on the fuction that is going to compare set_common_genes and the ids found in the CLIP files. 
#feature[9] example --> 'ENSG00000107521.20|HPS1|mRNA;ENSG00000264610.1|MIR4685|miRNA'
for features in clipfile: 
    feature[9].to_df --> split with read_bindingsites_4fields(path: str) -> pd.DataFrame: --> df['gene_id', 'name', 'type']
    if df.gene_id.isin(set_common_genes) : 
        list.append(feature) 

After processing the miCLIP files, the eCLIP files are to be manipulated. For eCLIP, the data is more consistent and collected in a directory, divided by protein (?).      

In [55]:
#accessing all of the files in a directory 
path = 'processed/HNRNPC_HepG2'#later to be changed to access the files in 'processed'
directory = os.listdir(path)
for file in directory :
    print(file)
    if 'peaks.crosslink.anno.bed' in file : #there's where the gene ids are, do we need the rest of the files? 
        #here we can add the fuction to compare the genes. --> probably needs to be modified.--> nope it should work anyway  
        eCLIP = BedTool(path +'/'+ file)
        df_eCLIP = eCLIP.to_dataframe(disable_auto_names=True, header= None)
        df_eCLIP['gene'] = df_eCLIP[9]
        df_eCLIP['in intersection'] =  df_eCLIP.gene.map(lambda x: compare_with_common_genes(x, genes_to_compare=set_common_genes))
        df_eclip_ok = df_miCLIP[df_miCLIP['in intersection'] == True] #I should save these df_eclip_ok, in which way should I do it?
        #maybe I can do another dataframe containing all og the OK rows but it would be very large if I search in all of the directory 
        

peaks.crosslink.bed
peaks.crosslink.anno.bed
peaks.bed


In [57]:
len(df_eCLIP)

19965

In [56]:
len(df_eclip_ok)

9087

In [7]:
#apparently, there are no gene ids in this file, right?? - yes, just the range of positions 
df2 = pd.read_csv("C:/Users/sofia/Desktop/THESIS/PROJECT/GSE63753_hek293.abcam.CIMS.m6A.9536.bed.txt", sep='\t', header = None)
df2.head()

,0,1,2,3,4,5,6,7,8
0,chr10,100176311,100176312,B045_1-4_chr10_r_c635[k=4][m=2]_AGACT,2,-,AGACT,4,0.500000
1,chr10,101090591,101090592,B045_1-4_chr10_f_c695[k=4][m=2]_GGACT,2,+,GGACT,4,0.500000
2,chr10,101370996,101370997,B045_1-4_chr10_r_c643[k=4][m=2]_GAACG,2,-,GAACG,4,0.500000
3,chr10,101503020,101503021,B045_1-4_chr10_f_c698[k=4][m=2]_TGACA,2,+,TGACA,4,0.500000
4,chr10,101515460,101515461,B045_1-4_chr10_f_c699[k=11][m=2]_GGACT,2,+,GGACT,11,0.181818
